In [ ]:
import pandas as pd
import numpy as np

In [ ]:
playerplays = pd.read_csv("player_play.csv")
plays = pd.read_csv("plays.csv")
players = pd.read_csv("players.csv")

In [ ]:
#Combine all tracking data
df = pd.read_csv("tracking_week_1.csv")
df2 = pd.read_csv("tracking_week_2.csv")
df3 = pd.read_csv("tracking_week_3.csv")
df4 = pd.read_csv("tracking_week_4.csv")
df5 = pd.read_csv("tracking_week_5.csv")
df6 = pd.read_csv("tracking_week_6.csv")
df7 = pd.read_csv("tracking_week_7.csv")
df8 = pd.read_csv("tracking_week_8.csv")
df9 = pd.read_csv("tracking_week_9.csv")
dfs = [df, df2, df3, df4, df5, df6, df7, df8, df9]
df = pd.concat(dfs, axis=0, ignore_index=True)
snappositions = df[['gameId','playId','nflId','y']][df['frameType']=="SNAP"]

In [ ]:
#Merge all necessary information for each player
motionplayers = playerplays[['gameId','playId','motionSinceLineset']][playerplays['motionSinceLineset']==True]
motionplays = pd.merge(
    motionplayers,
    plays[['gameId','playId','passResult']],
    how = "left",
    on = ['gameId','playId']
)
motionplays = motionplays[motionplays['passResult'].notna()]
routeRunners = pd.merge(
    playerplays[['gameId','playId','nflId','motionSinceLineset','inMotionAtBallSnap','wasRunningRoute','wasTargettedReceiver']][playerplays['wasRunningRoute']==True],
    plays[['gameId','playId','down','yardsToGo','offenseFormation','receiverAlignment','pff_passCoverage','absoluteYardlineNumber',]],
    how = "left",
    left_on = ['gameId','playId'],
    right_on = ['gameId','playId']
)

motionRunners = routeRunners.merge(
    motionplays[['gameId', 'playId']].drop_duplicates(), 
    on=['gameId', 'playId'], 
    how='inner'
)
motionRunners = pd.merge(
    motionRunners,
    snappositions,
    how = "left",
    on = ['gameId','playId','nflId']
)

In [ ]:
#Only find plays with motion runners
allWeeks = motionRunners[motionRunners['y'].notna()]
allWeeks['motionSinceLineset'] = allWeeks['motionSinceLineset'].fillna(False)
allWeeks['motionSinceLineset'] = allWeeks['motionSinceLineset'].fillna(False)

In [ ]:
#Find coordinates of players at ball snap
linecoords = pd.read_csv("linecoods.csv")
linecoords2 = pd.read_csv("linecoods2.csv")
linecoords3 = pd.read_csv("linecoods3.csv")
linecoords4 = pd.read_csv("linecoods4.csv")
linecoords5 = pd.read_csv("linecoods5.csv")
linecoords6 = pd.read_csv("linecoods6.csv")
linecoords7 = pd.read_csv("linecoods7.csv")
linecoords8 = pd.read_csv("linecoods8.csv")
linecoords9 = pd.read_csv("linecoods9.csv")
linecoords = pd.concat([linecoords,linecoords2,linecoords3,linecoords4,linecoords5,linecoords6,linecoords7,linecoords8,linecoords9],)

In [ ]:
#Add additional play information
allWeeks = pd.merge(
    allWeeks,
    linecoords[['gameId','playId','topOfLine','bottomOfLine']],
    how = "left",
    on = ['gameId','playId']
    )
allWeeks = pd.merge(
    allWeeks,
    players[['nflId','position']]
)
safetyPositions = pd.read_csv("safetyPositions")
allWeeks = pd.merge(
    allWeeks,
    safetyPositions,
    how = "left",
    on = ['gameId','playId']
)
allWeeks = allWeeks[allWeeks['topOfLine'].notna()]

In [ ]:
allWeeks.to_csv("allVoronoi.csv")

In [ ]:
#Flatten each play into one data point
all = pd.DataFrame(columns=[
    'gameId', 'playId', 'down', 'yardsToGo', 'offenseFormation', 
    'pff_passCoverage', 'absoluteYardlineNumber', 'receiverAlignment',
    'firstY', 'secondY', 'thirdY', 'fourthY', 'fifthY', 'sixthY', 'safetyTop','safetyBottom', 'rbPosition',
    'aboveLine', 'belowLine','motionReceiver', 'motionCluster', 'inMotionAtBallSnap', 
    'targetedReceiver', 'maxSeparationReceiver', 'nflId'
])

for _, i in allWeeks.iterrows():
    inone = ((one['gameId'] == i['gameId']) & (one['playId'] == i['playId'])).any()
    
    if not inone:
        play = allWeeks[(allWeeks['gameId'] == i['gameId']) & 
                       (allWeeks['playId'] == i['playId'])]
        play = play.sort_values(by='y', ascending=False)
        motion_players = play[play['motionSinceLineset']]  # Get motion players

        for _, motion_player in motion_players.iterrows():
            thisplay = [
                i['gameId'],
                i['playId'],
                i['down'], 
                i['yardsToGo'], 
                i['offenseFormation'],
                i['pff_passCoverage'], 
                i['absoluteYardlineNumber'],
                i['receiverAlignment']
            ]

            motionrec = 0
            targetrec = 0
            motionindex = 1
            targetindex = 1
            aboveline = 0
            belowline = 0
            moving = False
            rbindex = 0
            for _, j in play.iterrows():
                thisplay.append(j['y'])
                if j['y'] > j['topOfLine']:
                    aboveline += 1
                elif j['y'] < j['bottomOfLine']:
                    belowline += 1
                if j['motionSinceLineset']:
                    if j['nflId'] == motion_player['nflId']:  # Focus on specific motion player
                        motionrec = motionindex
                        if j['inMotionAtBallSnap']:
                            moving = True
                if j['position'] == 'RB':
                    rbindex = targetindex
                if j['wasTargettedReceiver']:
                    targetrec = targetindex
                motionindex += 1
                targetindex += 1

            while len(thisplay) < 14:
                thisplay.append(0)

            thisplay.extend([
                i['safetyTop'],
                i['safetyBottom'],
                rbindex,
                aboveline,
                belowline,
                motionrec,
                0,  # Add motion cluster
                moving,
                targetrec,
                0,  # Find max separation receiver
                motion_player['nflId']
            ])
            
            new_row = pd.DataFrame([thisplay], columns=one.columns)
            one = pd.concat([one, new_row], ignore_index=True)

In [ ]:
#Add cluster infomration and filter out nans
clusters = pd.read_csv('clusters.csv')
all_weeks = pd.merge(
    one,
    clusters[['gameId','playId','nflId','cluster']],
    how = "left",
    on = ['gameId','playId', 'nflId']
)
all_weeks['motionCluster'] = all_weeks['cluster']
all_weeks = all_weeks.dropna(subset=['motionCluster'])
all_weeks = all_weeks[all_weeks['targetedReceiver']!=0]
all_weeks['safetyBottom'] = all_weeks['safetyBottom'].fillna(0)
all_weeks['safetyTop'] = all_weeks['safetyTop'].fillna(0)

all_weeks = all_weeks.drop(columns = ['cluster', 'nflId'])
all_weeks.to_csv("allWeekMotionPlays")